In [17]:
!pip install --upgrade tensorflow 2.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement 2.9 (from versions: none)
ERROR: No matching distribution found for 2.9


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [6]:
df = pd.read_csv("/insurance_data.csv")

In [7]:
df.head(5)
new_df=df.copy()
new_df['age']=new_df['age']/100

**USING TENSORFLOW**

In [8]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest = train_test_split(new_df[['age','affordibility']],new_df['bought_insurance'])

In [9]:
Xtrain.head(5)

,age,affordibility
13,0.29,0
20,0.21,1
18,0.19,0
23,0.45,1
7,0.60,0


In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',
    bias_initializer='zeros')
])
"""
Optimizer adam has it's own learning rate to change it we need to use change by defining the object of this
"""
opti = tf.keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=opti,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(Xtrain,ytrain,epochs=5100)


Epoch 1/5100
1/1 [==============================] - 1s 985ms/step - loss: 0.7334 - accuracy: 0.4762
Epoch 2/5100
1/1 [==============================] - 0s 10ms/step - loss: 0.7330 - accuracy: 0.4762
Epoch 3/5100
1/1 [==============================] - 0s 12ms/step - loss: 0.7325 - accuracy: 0.4762
Epoch 4/5100
1/1 [==============================] - 0s 11ms/step - loss: 0.7321 - accuracy: 0.4762
Epoch 5/5100
1/1 [==============================] - 0s 11ms/step - loss: 0.7317 - accuracy: 0.4762
Epoch 6/5100
1/1 [==============================] - 0s 10ms/step - loss: 0.7312 - accuracy: 0.4762
Epoch 7/5100
1/1 [==============================] - 0s 10ms/step - loss: 0.7308 - accuracy: 0.4762
Epoch 8/5100
1/1 [==============================] - 0s 10ms/step - loss: 0.7303 - accuracy: 0.4762
Epoch 9/5100
1/1 [==============================] - 0s 15ms/step - loss: 0.7299 - accuracy: 0.4762
Epoch 10/5100
1/1 [==============================] - 0s 11ms/step - loss: 0.7294 - accuracy: 0.4762
Epoch 11

In [266]:
display(model.predict(Xtest))
display(ytest)
model.evaluate(Xtest,ytest)

1/1 [==============================] - 0s 50ms/step


array([[0.34394038],
       [0.631705  ],
       [0.7937553 ],
       [0.16897206],
       [0.39406177],
       [0.4789277 ],
       [0.5461315 ]], dtype=float32)

10    0
22    1
15    1
12    0
0     0
6     0
7     1
Name: bought_insurance, dtype: int64

1/1 [==============================] - 0s 131ms/step - loss: 0.4364 - accuracy: 1.0000


[0.43637824058532715, 1.0]

In [267]:
def sigmoid(y):
  type(y)
  return 1/(1+math.exp(-y))

In [268]:
def prediction_function(X,model):
  coef,intercept=model.get_weights()
  y=X@coef+intercept[0]
  y=[sigmoid(val) for val in y.values]
  return y

In [269]:
  coef,intercept=model.get_weights()
  coef.T.shape,Xtest.shape
  y=Xtest*coef.T+intercept[0]
  # print(y.values)
  prediction_function(Xtest,model)

[0.34394037077094736,
 0.6317049664211167,
 0.7937552956812995,
 0.16897203861021176,
 0.39406176893687783,
 0.47892769192136087,
 0.546131445555281]

**FROM SCRATCH**

In [270]:
class NeuralNetwork:
  def __init__(self,X,y):
    self.X=X
    self.y=y

  def sigmoid(self,y):
    return 1/(1+np.exp(-y))

  def log_loss(self,ytrue,ypredict):
    """
    Underflow or Overflow: The calculations involving very small or very large numbers can result in numerical instability, leading to infinity or negative infinity.
    This is not mathematical error but hardware error since it cannot calculate precisely.

    Add Epsilon: To avoid taking the logarithm of exactly 0 or 1, you can add a small epsilon value to the predictions before calculating the log loss.
    he np.clip() function is used to limit the values in an array to a specified range. It ensures that the values in the array are within the given bounds.
    The syntax of the np.clip() : np.clip(a, a_min, a_max, out=None)

    """
    epsilon = 1e-15
    ypredict = np.clip(ypredict, epsilon, 1 - epsilon)
    log_loss_value = -np.mean(ytrue * np.log(ypredict) + (1 - ytrue) * np.log(1 - ypredict))
    return log_loss_value

  def gradient_descent(self,X,y,alpha,epochs=100,display=False):
    """
    Using vectorization to take advantage of numpy
    Step1: Make y predict by y=w1*x1+w2*x2+....+wn*xn+b
    Step2: Use y predict in activation function sigmoid(y)
    Step3: Calculate dw = sum(x_i*(y-ypredict))/rows_in_X
                     db = sum(y-ypredict)/rows_in_X
    Step4: Update w and b
    Step5: Repeat this until convergence or for specified no. of epochs
    """
    w=np.ones(len(X.columns))
    print(y.shape)
    b=0
    for _ in range(epochs):
      ypredict=self.sigmoid(w@X.values.T+b)
      loss=self.log_loss(y,ypredict)
      y_diff=np.array(ytrain-ypredict)
      dw=-alpha*(X.T@y_diff)/X.shape[1]
      db=-alpha*np.mean(y_diff)
      if display==True:
        print(f"Epochs={_},  loss={loss}")
      w-=dw
      b-=db
    return (w,b)

  def fit(self,alpha=0.05,epochs=100,display=True):
    self.weights,self.biases=self.gradient_descent(self.X,self.y,alpha,epochs,display)
    print("Weights = ",self.weights)
    print("self.biases = ",self.biases)

  def predict(self,Xtest):
    """
    Based on the calculated weights and biases make predictions
    """
    return self.sigmoid(self.weights@Xtest.values.T+self.biases)

In [277]:
model2=NeuralNetwork(Xtrain,ytrain)
model2.fit(0.01,5100,True)

Streaming output truncated to the last 5000 lines.
Epochs=104,  loss=0.6308542011417343
Epochs=105,  loss=0.6307262591777212
Epochs=106,  loss=0.630599554672155
Epochs=107,  loss=0.6304740390065773
Epochs=108,  loss=0.6303496655384301
Epochs=109,  loss=0.6302263895236975
Epochs=110,  loss=0.6301041680423655
Epochs=111,  loss=0.629982959926617
Epochs=112,  loss=0.6298627256916716
Epochs=113,  loss=0.6297434274691882
Epochs=114,  loss=0.6296250289431433
Epochs=115,  loss=0.6295074952881071
Epochs=116,  loss=0.6293907931098345
Epochs=117,  loss=0.6292748903880954
Epochs=118,  loss=0.6291597564216693
Epochs=119,  loss=0.6290453617754286
Epochs=120,  loss=0.6289316782294385
Epochs=121,  loss=0.6288186787300065
Epochs=122,  loss=0.628706337342612
Epochs=123,  loss=0.6285946292066479
Epochs=124,  loss=0.628483530491914
Epochs=125,  loss=0.6283730183567972
Epochs=126,  loss=0.628263070908079
Epochs=127,  loss=0.6281536671623114
Epochs=128,  loss=0.6280447870087054
Epochs=129,  loss=0.627936411

In [278]:
print(model2.predict(Xtest))
print(model.predict(Xtest))


[0.31902051 0.62980627 0.80387652 0.23813414 0.37196581 0.61738235
 0.68385771]
1/1 [==============================] - 0s 29ms/step
[[0.34394038]
 [0.631705  ]
 [0.7937553 ]
 [0.16897206]
 [0.39406177]
 [0.4789277 ]
 [0.5461315 ]]



**Almost close**

In [276]:
print("With Tensorflow = ")
print("age    ",coef[0][0],"\n",
      "affordibility ",coef[1][0])
print("bias = ",intercept[0])

With Tensorflow = 
age     5.387825 
 affordibility  1.4320511
bias =  -3.047643
